# MODELADO BASELINE - RANDOM FOREST

Objetivo: Establecer un segundo modelo baseline usando Random Forest para comparar con Regresión Logística y los modelos de Gradient Boosting.

- Es un modelo ensemble robusto y ampliamente usado
- No requiere escalado de variables
- Maneja bien variables categóricas codificadas
- Proporciona importancia de features nativa
- Sirve como punto intermedio entre RL (lineal) y Gradient Boosting (más complejo)

Fases temporales:
- T0 (Matrícula)        : Variables disponibles al momento de inscripción
- T1 (Fin 1er Semestre) : T0 + variables académicas del 1er semestre
- T2 (Fin 2do Semestre) : T1 + variables académicas del 2do semestre

Preprocesamiento para Random Forest:
- NO requiere escalado
- Label Encoding para categóricas
- Target Encoding para 'course' (alta cardinalidad)
- class_weight='balanced' para desbalance

## 0. Librerias y configuración

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Preprocesamiento
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import LabelEncoder

# Modelo
from sklearn.ensemble import RandomForestClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, precision_recall_curve, average_precision_score
)

# Target Encoding
from category_encoders import TargetEncoder

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Seed para reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Directorio de salida
OUTPUT_DIR = "../outputs/figures/modelado/baseline_RF/"
OUTPUT_DIR_REPORTES = "../outputs/models/baseline_RF/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR_REPORTES, exist_ok=True)

# mlflow
import mlflow
import mlflow.sklearn

## 1. Carga de datos preprocesados

In [18]:
# Cargar dataset preprocesado
df = pd.read_csv('../data/processed/preprocessed_data.csv')

print(f"Dataset cargado: {df.shape[0]} filas x {df.shape[1]} columnas")
print(df['target_binario'].value_counts())
print(f"\nRatio de desbalance: {df['target_binario'].value_counts()[0] / df['target_binario'].value_counts()[1]:.2f}:1")

df.head()

Dataset cargado: 4424 filas x 38 columnas
target_binario
0    3003
1    1421
Name: count, dtype: int64

Ratio de desbalance: 2.11:1


,application_order,course,daytimeevening_attendance,previous_qualification_grade,admission_grade,displaced,educational_special_needs,debtor,tuition_fees_up_to_date,gender,scholarship_holder,age_at_enrollment,international,curricular_units_1st_sem_credited,curricular_units_1st_sem_enrolled,curricular_units_1st_sem_evaluations,curricular_units_1st_sem_approved,curricular_units_1st_sem_grade,curricular_units_1st_sem_without_evaluations,curricular_units_2nd_sem_credited,curricular_units_2nd_sem_enrolled,curricular_units_2nd_sem_evaluations,curricular_units_2nd_sem_approved,curricular_units_2nd_sem_grade,curricular_units_2nd_sem_without_evaluations,unemployment_rate,inflation_rate,gdp,is_single,application_mode_risk,is_over_23_entry,previous_qualification_risk,mothers_qualification_level,fathers_qualification_level,mothers_occupation_level,fathers_occupation_level,has_unknown_parent_info,target_binario
0,5,171,1,122.0000,127.3000,1,0,0,1,1,0,20,0,0,0,0,0,0.0000,0,0,0,0,0,0.0000,0,10.8000,1.4000,1.7400,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Secundaria,Otro_Trabajo,Otro_Trabajo,0,1
1,1,9254,1,160.0000,142.5000,1,0,0,0,1,0,19,0,0,6,6,6,14.0000,0,0,6,6,6,13.6667,0,13.9000,-0.3000,0.7900,1,Bajo_Riesgo,0,Bajo_Riesgo,Secundaria,Superior,Profesional,Profesional,0,0
2,5,9070,1,122.0000,124.8000,1,0,0,0,1,0,19,0,0,6,0,0,0.0000,0,0,6,0,0,0.0000,0,10.8000,1.4000,1.7400,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Baja,Basica_Baja,Otro_Trabajo,Otro_Trabajo,0,1
3,2,9773,1,122.0000,119.6000,1,0,0,1,0,0,20,0,0,6,8,6,13.4286,0,0,6,10,5,12.4000,0,9.4000,-0.8000,-3.1200,1,Bajo_Riesgo,0,Bajo_Riesgo,Basica_Media,Basica_Baja,Otro_Trabajo,Profesional,0,0
4,1,8014,0,100.0000,141.5000,0,0,0,1,0,0,45,0,0,6,9,5,12.3333,0,0,6,6,6,13.0000,0,13.9000,-0.3000,0.7900,0,Alto_Riesgo,1,Bajo_Riesgo,Basica_Baja,Basica_Media,Otro_Trabajo,Otro_Trabajo,0,0


## 2. Definición de variables por fase temporal (T0, T1, T2)

In [19]:
# =============================================================================
# DEFINICIÓN DE VARIABLES POR TIPO
# =============================================================================

# TARGET
TARGET = 'target_binario'

# -----------------------------------------------------------------------------
# VARIABLES BINARIAS (no requieren encoding, ya son 0/1)
# -----------------------------------------------------------------------------
VARS_BINARIAS_T0 = [
    'daytimeevening_attendance',
    'displaced',
    'educational_special_needs',
    'gender',
    'scholarship_holder',
    'international',
    'is_single'
]

VARS_BINARIAS_T1 = [
    'debtor',
    'tuition_fees_up_to_date'
]

# -----------------------------------------------------------------------------
# VARIABLES NUMÉRICAS (NO requieren escalado para RF)
# -----------------------------------------------------------------------------
VARS_NUMERICAS_T0 = [
    'age_at_enrollment',
    'admission_grade',
    'previous_qualification_grade'
]

VARS_NUMERICAS_T1 = [
    'curricular_units_1st_sem_credited',
    'curricular_units_1st_sem_enrolled',
    'curricular_units_1st_sem_evaluations',
    'curricular_units_1st_sem_approved',
    'curricular_units_1st_sem_grade',
    'curricular_units_1st_sem_without_evaluations',
    'unemployment_rate',
    'inflation_rate',
    'gdp'
]

VARS_NUMERICAS_T2 = [
    'curricular_units_2nd_sem_credited',
    'curricular_units_2nd_sem_enrolled',
    'curricular_units_2nd_sem_evaluations',
    'curricular_units_2nd_sem_approved',
    'curricular_units_2nd_sem_grade',
    'curricular_units_2nd_sem_without_evaluations'
]

# -----------------------------------------------------------------------------
# VARIABLES CATEGÓRICAS AGRUPADAS (requieren Label Encoding)
# -----------------------------------------------------------------------------
VARS_CATEGORICAS_AGRUPADAS_T0 = [
    'application_mode_risk',
    'previous_qualification_risk',
    'mothers_qualification_level',
    'fathers_qualification_level',
    'mothers_occupation_level',
    'fathers_occupation_level'
]

# -----------------------------------------------------------------------------
# VARIABLES CATEGÓRICAS PARA TARGET ENCODING
# -----------------------------------------------------------------------------
VARS_TARGET_ENCODING_T0 = ['course']

# -----------------------------------------------------------------------------
# VARIABLE ORDINAL (se trata como numérica)
# -----------------------------------------------------------------------------
VARS_ORDINALES_T0 = ['application_order']

# =============================================================================
# COMPOSICIÓN DE VARIABLES POR FASE TEMPORAL
# =============================================================================

# T0: Variables disponibles al momento de matrícula
VARS_T0 = (
    VARS_BINARIAS_T0 +
    VARS_NUMERICAS_T0 +
    VARS_CATEGORICAS_AGRUPADAS_T0 +
    VARS_TARGET_ENCODING_T0 +
    VARS_ORDINALES_T0
)

# T1: T0 + variables del 1er semestre
VARS_T1 = (
    VARS_T0 +
    VARS_BINARIAS_T1 +
    VARS_NUMERICAS_T1
)

# T2: T1 + variables del 2do semestre
VARS_T2 = (
    VARS_T1 +
    VARS_NUMERICAS_T2
)

print("================================================================================")
print("  VARIABLES POR FASE TEMPORAL")
print("================================================================================")
print(f"\n T0 (Matrícula): {len(VARS_T0)} variables")
print(f" T1 (Fin 1er Sem): {len(VARS_T1)} variables (+{len(VARS_T1) - len(VARS_T0)})")
print(f" T2 (Fin 2do Sem): {len(VARS_T2)} variables (+{len(VARS_T2) - len(VARS_T1)})")

  VARIABLES POR FASE TEMPORAL

 T0 (Matrícula): 18 variables
 T1 (Fin 1er Sem): 29 variables (+11)
 T2 (Fin 2do Sem): 35 variables (+6)


## 3. Split TRAIN/TEST 

In [20]:
# =============================================================================
# SPLIT ESTRATIFICADO 80/20
# =============================================================================

X = df[VARS_T2].copy()
y = df[TARGET].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("================================================================================")
print("  SPLIT TRAIN/TEST")
print("================================================================================")
print(f"\nTrain: {X_train.shape[0]} ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Test:  {X_test.shape[0]} ({X_test.shape[0]/len(df)*100:.1f}%)")

print(f"\nDistribución del target en Train:")
print(y_train.value_counts())
print(f"Ratio de desbalance:: {y_train.value_counts()[0] / y_train.value_counts()[1]:.2f}:1")

print(f"\nDistribución del target en Test:")
print(y_test.value_counts())
print(f"Ratio de desbalance:: {y_test.value_counts()[0] / y_test.value_counts()[1]:.2f}:1")

  SPLIT TRAIN/TEST

Train: 3539 (80.0%)
Test:  885 (20.0%)

Distribución del target en Train:
target_binario
0    2402
1    1137
Name: count, dtype: int64
Ratio de desbalance:: 2.11:1

Distribución del target en Test:
target_binario
0    601
1    284
Name: count, dtype: int64
Ratio de desbalance:: 2.12:1


## 4. Funciones de preprocesamiento para RANDOM FOREST

In [21]:
def obtiene_variables_por_fase(fase):
    """
    Retorna las listas de variables según la fase temporal.
    """
    if fase == 'T0':
        return {
            'binarias': VARS_BINARIAS_T0,
            'numericas': VARS_NUMERICAS_T0 + VARS_ORDINALES_T0,
            'categoricas_le': VARS_CATEGORICAS_AGRUPADAS_T0,
            'categoricas_te': VARS_TARGET_ENCODING_T0,
            'all': VARS_T0
        }
    elif fase == 'T1':
        return {
            'binarias': VARS_BINARIAS_T0 + VARS_BINARIAS_T1,
            'numericas': VARS_NUMERICAS_T0 + VARS_ORDINALES_T0 + VARS_NUMERICAS_T1,
            'categoricas_le': VARS_CATEGORICAS_AGRUPADAS_T0,
            'categoricas_te': VARS_TARGET_ENCODING_T0,
            'all': VARS_T1
        }
    elif fase == 'T2':
        return {
            'binarias': VARS_BINARIAS_T0 + VARS_BINARIAS_T1,
            'numericas': VARS_NUMERICAS_T0 + VARS_ORDINALES_T0 + VARS_NUMERICAS_T1 + VARS_NUMERICAS_T2,
            'categoricas_le': VARS_CATEGORICAS_AGRUPADAS_T0,
            'categoricas_te': VARS_TARGET_ENCODING_T0,
            'all': VARS_T2
        }
    else:
        raise ValueError(f"Fase no válida: {fase}. Usar 'T0', 'T1', o 'T2'")


def preprocesamiento_RF(X_train, X_test, y_train, fase):
    # Preprocesa los datos para Random Forest    
    
    variables_fase = obtiene_variables_por_fase(fase)
    
    # Seleccionar solo las variables de la fase
    X_train_fase = X_train[variables_fase['all']].copy()
    X_test_fase = X_test[variables_fase['all']].copy()
    
    # Diccionario para guardar encoders
    label_encoders = {}
    
    # -------------------------------------------------------------------------
    # 1. TARGET ENCODING para 'course' 
    # -------------------------------------------------------------------------
    te = TargetEncoder(cols=variables_fase['categoricas_te'], smoothing=0.3)
    
    for col in variables_fase['categoricas_te']:
        X_train_fase[col + '_encoded'] = te.fit_transform(X_train_fase[[col]], y_train)[col]
        X_test_fase[col + '_encoded'] = te.transform(X_test_fase[[col]])[col]
        # Eliminar columna original
        X_train_fase = X_train_fase.drop(columns=[col])
        X_test_fase = X_test_fase.drop(columns=[col])
    
    # -------------------------------------------------------------------------
    # 2. LABEL ENCODING para categóricas agrupadas
    # -------------------------------------------------------------------------
    for col in variables_fase['categoricas_le']:
        le = LabelEncoder()
        X_train_fase[col] = le.fit_transform(X_train_fase[col].astype(str))
        X_test_fase[col] = le.transform(X_test_fase[col].astype(str))
        label_encoders[col] = le
    
    # -------------------------------------------------------------------------
    # Guardar información
    # -------------------------------------------------------------------------
    feature_names = X_train_fase.columns.tolist()
    preprocessors = {
        'target_encoder': te,
        'label_encoders': label_encoders,
        'feature_names': feature_names
    }
    
    return X_train_fase, X_test_fase, feature_names, preprocessors


## 5. Funciones entrenamiento

In [22]:
def entrena_RF(X_train, y_train, fase, cv_folds=5):
    """Entrena y evalúa Random Forest con Cross-Validation."""

    mlflow.end_run()
      
    print("================================================================================")
    print(f"  ENTRENAMIENTO Random Forest - FASE {fase}")
    print("================================================================================")
    print(f"\nVariables: {X_train.shape[1]}")
    print(f"Registros: {X_train.shape[0]}")
    
    print(f"\nHiperparámetros (por defecto):")
    print(f"   • n_estimators: 100")
    print(f"   • max_depth: None (sin límite)")
    print(f"   • min_samples_split: 2")
    print(f"   • min_samples_leaf: 1")
    print(f"   • max_features: sqrt")
    print(f"   • class_weight: balanced")
    
    # -------------------------------------------------------------------------
    # Cross-Validation con loop manual
    # -------------------------------------------------------------------------
    print(f"\nCross-Validation ({cv_folds}-fold):")
    
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE)
    
    # Almacenar resultados por fold
    cv_results = {
        'train_accuracy': [], 'test_accuracy': [],
        'train_precision': [], 'test_precision': [],
        'train_recall': [], 'test_recall': [],
        'train_f1': [], 'test_f1': [],
        'train_roc_auc': [], 'test_roc_auc': []
    }
    
    oob_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        X_fold_train = X_train.iloc[train_idx]
        X_fold_val = X_train.iloc[val_idx]
        y_fold_train = y_train.iloc[train_idx]
        y_fold_val = y_train.iloc[val_idx]
        
        # Crear y entrenar modelo
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            max_features='sqrt',
            bootstrap=True,
            class_weight='balanced',
            random_state=RANDOM_STATE,
            n_jobs=-1,
            oob_score=True
        )
        model.fit(X_fold_train, y_fold_train)
        
        # Guardar OOB score del fold
        oob_scores.append(model.oob_score_)
        
        # Predicciones
        y_train_pred = model.predict(X_fold_train)
        y_train_proba = model.predict_proba(X_fold_train)[:, 1]
        y_val_pred = model.predict(X_fold_val)
        y_val_proba = model.predict_proba(X_fold_val)[:, 1]
        
        # Métricas Train
        cv_results['train_accuracy'].append(accuracy_score(y_fold_train, y_train_pred))
        cv_results['train_precision'].append(precision_score(y_fold_train, y_train_pred))
        cv_results['train_recall'].append(recall_score(y_fold_train, y_train_pred))
        cv_results['train_f1'].append(f1_score(y_fold_train, y_train_pred))
        cv_results['train_roc_auc'].append(roc_auc_score(y_fold_train, y_train_proba))
        
        # Métricas Validation
        cv_results['test_accuracy'].append(accuracy_score(y_fold_val, y_val_pred))
        cv_results['test_precision'].append(precision_score(y_fold_val, y_val_pred))
        cv_results['test_recall'].append(recall_score(y_fold_val, y_val_pred))
        cv_results['test_f1'].append(f1_score(y_fold_val, y_val_pred))
        cv_results['test_roc_auc'].append(roc_auc_score(y_fold_val, y_val_proba))
    
    # Convertir a numpy arrays
    for key in cv_results:
        cv_results[key] = np.array(cv_results[key])
    
    # -------------------------------------------------------------------------
    # Resultados por fold
    # -------------------------------------------------------------------------
    print("\n Resultados por fold:")
    for i in range(cv_folds):
        print(f"\n  Fold {i+1}:")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
            train_score = cv_results[f'train_{metric}'][i]
            val_score = cv_results[f'test_{metric}'][i]
            print(f"    {metric:<10} | Train: {train_score:.4f} | Val: {val_score:.4f}")

    mlflow.set_experiment("TFM_Dropout_Prediction")
    with mlflow.start_run(run_name=f"RandomForest_CV5_{fase}"):
        mlflow.set_tag("modelo", 'Baseline - Params por default')
        mlflow.set_tag("tipo", 'Validacion cruzada')
        mlflow.log_params(model.get_params())

        # -------------------------------------------------------------------------
        # Resumen CV (media ± std)
        # -------------------------------------------------------------------------
        print(f"\n Resumen Cross-Validation:")
        print(f"\n   {'Métrica':<12} {'Train Mean':>12} {'Train Std':>12} {'Val Mean':>12} {'Val Std':>12}")
        print(f"   {'-'*60}")
        for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
            train_mean = cv_results[f'train_{metric}'].mean()
            train_std = cv_results[f'train_{metric}'].std()
            val_mean = cv_results[f'test_{metric}'].mean()
            val_std = cv_results[f'test_{metric}'].std()
            # mlflow
            mlflow.log_metric(f'test_{metric}_mean', val_mean.round(4))
            mlflow.log_metric(f'test_{metric}_std', val_std.round(4))
                
            print(f"   {metric:<12} {train_mean:>12.4f} {train_std:>12.4f} {val_mean:>12.4f} {val_std:>12.4f}")
    
    
    # -------------------------------------------------------------------------
    # Retornar resultados
    # -------------------------------------------------------------------------
    results = {
        'phase': fase,
        'model': model,
        'n_features': X_train.shape[1],
        'cv_results': cv_results,
        'oob_score': model.oob_score_,
        'oob_scores_cv': oob_scores
    }
    
    return results



def resumen_cv(cv_results, fase, modelo):
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

    summary = {
        'modelo': modelo,
        'fase': fase
    }

    # -------------------------
    # Métricas de VALIDACIÓN
    # -------------------------
    for metric in metrics:
        summary[f'{metric}_val_mean'] = cv_results[f'test_{metric}'].mean()
        summary[f'{metric}_val_std']  = cv_results[f'test_{metric}'].std()
        

    # -------------------------
    # Métricas de TRAIN
    # -------------------------
    for metric in metrics:
        summary[f'{metric}_train_mean'] = cv_results[f'train_{metric}'].mean()
        summary[f'{metric}_train_std']  = cv_results[f'train_{metric}'].std()

    return pd.DataFrame([summary])


## 6. Modelado FASE T0 (MATRÍCULA)

In [23]:
# Preprocesamiento para T0
X_train_T0, X_test_T0, features_T0, prep_T0 = preprocesamiento_RF(
    X_train, X_test, y_train, fase='T0'
)

print(f"\n T0 - Dimensiones después del preprocesamiento:")
print(f"   Train: {X_train_T0.shape}")
print(f"   Test:  {X_test_T0.shape}")
print(f"   Features: {len(features_T0)}")
print(f"   \nLas variables son:")
X_train_T0.columns


 T0 - Dimensiones después del preprocesamiento:
   Train: (3539, 18)
   Test:  (885, 18)
   Features: 18
   
Las variables son:


Index(['daytimeevening_attendance', 'displaced', 'educational_special_needs',
       'gender', 'scholarship_holder', 'international', 'is_single',
       'age_at_enrollment', 'admission_grade', 'previous_qualification_grade',
       'application_mode_risk', 'previous_qualification_risk',
       'mothers_qualification_level', 'fathers_qualification_level',
       'mothers_occupation_level', 'fathers_occupation_level',
       'application_order', 'course_encoded'],
      dtype='object')

In [24]:
# Entrenar y evaluar T0
results_T0 = entrena_RF(X_train_T0, y_train, fase='T0')

df_resumen_rf = resumen_cv(
    cv_results=results_T0['cv_results'],
    fase='T0',
    modelo='RandomForest'
)
# Guardar tabla de comparación
df_resumen_rf.to_csv(f"{OUTPUT_DIR_REPORTES}cv_summary_RF.csv", index=False)

print(f"Resultados guardados en: {OUTPUT_DIR_REPORTES}cv_summary_RF.csv")

  ENTRENAMIENTO Random Forest - FASE T0

Variables: 18
Registros: 3539

Hiperparámetros (por defecto):
   • n_estimators: 100
   • max_depth: None (sin límite)
   • min_samples_split: 2
   • min_samples_leaf: 1
   • max_features: sqrt
   • class_weight: balanced

Cross-Validation (5-fold):

 Resultados por fold:

  Fold 1:
    accuracy   | Train: 1.0000 | Val: 0.7232
    precision  | Train: 1.0000 | Val: 0.6081
    recall     | Train: 1.0000 | Val: 0.3947
    f1         | Train: 1.0000 | Val: 0.4787
    roc_auc    | Train: 1.0000 | Val: 0.7427

  Fold 2:
    accuracy   | Train: 1.0000 | Val: 0.7316
    precision  | Train: 1.0000 | Val: 0.6439
    recall     | Train: 1.0000 | Val: 0.3728
    f1         | Train: 1.0000 | Val: 0.4722
    roc_auc    | Train: 1.0000 | Val: 0.7399

  Fold 3:
    accuracy   | Train: 1.0000 | Val: 0.7345
    precision  | Train: 1.0000 | Val: 0.6383
    recall     | Train: 1.0000 | Val: 0.3965
    f1         | Train: 1.0000 | Val: 0.4891
    roc_auc    | Train:

### Comentarios FASE 0

1. Se observa que todas las métricas en entrenamiento alcanzan valor máximo (1.0) lo que indica sobreentrenamiento severo. El recall es 0.3984 lo cual indica una baja capacidad de predicción.
2. Las métricas de validación caen de forma relevente, en consecuencia, no generaliza adecuadamente.
2. Estos resultados son inferioes a la RL, en consecuencia no es adecuado para predecir de forma temprana en esta fase.

## 7. Modelado FASE T1 (FIN 1ER SEMESTRE)

In [25]:
# Preprocesamiento para T1
X_train_T1, X_test_T1, features_T1, prep_T1 = preprocesamiento_RF(
    X_train, X_test, y_train, fase='T1'
)

print(f"\nT1 - Dimensiones después del preprocesamiento:")
print(f"   Train: {X_train_T1.shape}")
print(f"   Test:  {X_test_T1.shape}")
print(f"   Features: {len(features_T1)}")
print(f"   \nLas variables son:")
X_train_T1.columns


T1 - Dimensiones después del preprocesamiento:
   Train: (3539, 29)
   Test:  (885, 29)
   Features: 29
   
Las variables son:


Index(['daytimeevening_attendance', 'displaced', 'educational_special_needs',
       'gender', 'scholarship_holder', 'international', 'is_single',
       'age_at_enrollment', 'admission_grade', 'previous_qualification_grade',
       'application_mode_risk', 'previous_qualification_risk',
       'mothers_qualification_level', 'fathers_qualification_level',
       'mothers_occupation_level', 'fathers_occupation_level',
       'application_order', 'debtor', 'tuition_fees_up_to_date',
       'curricular_units_1st_sem_credited',
       'curricular_units_1st_sem_enrolled',
       'curricular_units_1st_sem_evaluations',
       'curricular_units_1st_sem_approved', 'curricular_units_1st_sem_grade',
       'curricular_units_1st_sem_without_evaluations', 'unemployment_rate',
       'inflation_rate', 'gdp', 'course_encoded'],
      dtype='object')

In [26]:
# Entrenar y evaluar T1
results_T1 = entrena_RF(X_train_T1, y_train, fase='T1')

df_resumen_rf_T1 = resumen_cv(
    cv_results=results_T1['cv_results'],
    fase='T1',
    modelo='RandomForest'
)

rf_path = "../outputs/models/baseline_RF/cv_summary_RF.csv"
df_rf = pd.read_csv(rf_path)
df_final = pd.concat([df_rf, df_resumen_rf_T1], ignore_index=True)

# Guardar tabla de comparación
df_final.to_csv(rf_path, index=False)
print(f"\nResultados guardados en: {OUTPUT_DIR_REPORTES}cv_summary_RF.csv")

  ENTRENAMIENTO Random Forest - FASE T1

Variables: 29
Registros: 3539

Hiperparámetros (por defecto):
   • n_estimators: 100
   • max_depth: None (sin límite)
   • min_samples_split: 2
   • min_samples_leaf: 1
   • max_features: sqrt
   • class_weight: balanced

Cross-Validation (5-fold):

 Resultados por fold:

  Fold 1:
    accuracy   | Train: 1.0000 | Val: 0.8588
    precision  | Train: 1.0000 | Val: 0.8404
    recall     | Train: 1.0000 | Val: 0.6930
    f1         | Train: 1.0000 | Val: 0.7596
    roc_auc    | Train: 1.0000 | Val: 0.8943

  Fold 2:
    accuracy   | Train: 1.0000 | Val: 0.8517
    precision  | Train: 1.0000 | Val: 0.8475
    recall     | Train: 1.0000 | Val: 0.6579
    f1         | Train: 1.0000 | Val: 0.7407
    roc_auc    | Train: 1.0000 | Val: 0.8797

  Fold 3:
    accuracy   | Train: 1.0000 | Val: 0.8418
    precision  | Train: 1.0000 | Val: 0.8010
    recall     | Train: 1.0000 | Val: 0.6740
    f1         | Train: 1.0000 | Val: 0.7321
    roc_auc    | Train:

### Comentarios FASE 1

1. Se mantiene las métricas perfectas de la fase 0, se mantiene en sobreentrenamiento.
2. Se observa una mejora con la incorporación de la Fase 0, con un recall de 0.6211 y AUC 0.9006 sin embargo, la caída de las métricas en validación es de importancia (prromerio 0.2), no generaliza de forma adecuada.
3. Aunque en rendimiento mejora, es inferior a la RL.

## 8. Modelado FASE T2 (FIN 2DO SEMESTRE)

In [27]:
# Preprocesamiento para T2
X_train_T2, X_test_T2, features_T2, prep_T2 = preprocesamiento_RF(
    X_train, X_test, y_train, fase='T2'
)

print(f"\nT2 - Dimensiones después del preprocesamiento:")
print(f"   Train: {X_train_T2.shape}")
print(f"   Test:  {X_test_T2.shape}")
print(f"   Features: {len(features_T2)}")
print(f"   \nLas variables son:")
X_train_T2.columns


T2 - Dimensiones después del preprocesamiento:
   Train: (3539, 35)
   Test:  (885, 35)
   Features: 35
   
Las variables son:


Index(['daytimeevening_attendance', 'displaced', 'educational_special_needs',
       'gender', 'scholarship_holder', 'international', 'is_single',
       'age_at_enrollment', 'admission_grade', 'previous_qualification_grade',
       'application_mode_risk', 'previous_qualification_risk',
       'mothers_qualification_level', 'fathers_qualification_level',
       'mothers_occupation_level', 'fathers_occupation_level',
       'application_order', 'debtor', 'tuition_fees_up_to_date',
       'curricular_units_1st_sem_credited',
       'curricular_units_1st_sem_enrolled',
       'curricular_units_1st_sem_evaluations',
       'curricular_units_1st_sem_approved', 'curricular_units_1st_sem_grade',
       'curricular_units_1st_sem_without_evaluations', 'unemployment_rate',
       'inflation_rate', 'gdp', 'curricular_units_2nd_sem_credited',
       'curricular_units_2nd_sem_enrolled',
       'curricular_units_2nd_sem_evaluations',
       'curricular_units_2nd_sem_approved', 'curricular_units_2nd

In [28]:
# Entrenar y evaluar T2
results_T2 = entrena_RF(X_train_T2, y_train, fase='T2')

df_resumen_rf_T2 = resumen_cv(
    cv_results=results_T2['cv_results'],
    fase='T2',
    modelo='RandomForest'
)

rf_path = "../outputs/models/baseline_RF/cv_summary_RF.csv"
df_rf = pd.read_csv(rf_path)
df_final = pd.concat([df_rf, df_resumen_rf_T2], ignore_index=True)
csv_path = "cv_summary_FL.csv"

# Guardar tabla de comparación
df_final.to_csv(rf_path, index=False)
print(f"\nResultados guardados en: {OUTPUT_DIR_REPORTES}cv_summary_RF.csv")

  ENTRENAMIENTO Random Forest - FASE T2

Variables: 35
Registros: 3539

Hiperparámetros (por defecto):
   • n_estimators: 100
   • max_depth: None (sin límite)
   • min_samples_split: 2
   • min_samples_leaf: 1
   • max_features: sqrt
   • class_weight: balanced

Cross-Validation (5-fold):

 Resultados por fold:

  Fold 1:
    accuracy   | Train: 1.0000 | Val: 0.8771
    precision  | Train: 1.0000 | Val: 0.8691
    recall     | Train: 1.0000 | Val: 0.7281
    f1         | Train: 1.0000 | Val: 0.7924
    roc_auc    | Train: 1.0000 | Val: 0.9074

  Fold 2:
    accuracy   | Train: 1.0000 | Val: 0.8686
    precision  | Train: 1.0000 | Val: 0.8729
    recall     | Train: 1.0000 | Val: 0.6930
    f1         | Train: 1.0000 | Val: 0.7726
    roc_auc    | Train: 1.0000 | Val: 0.9101

  Fold 3:
    accuracy   | Train: 1.0000 | Val: 0.8616
    precision  | Train: 1.0000 | Val: 0.8525
    recall     | Train: 1.0000 | Val: 0.6872
    f1         | Train: 1.0000 | Val: 0.7610
    roc_auc    | Train:

### Comentarios FASE 2

1. Se mantiene las métricas perfectas de la fase 0, el sobreentrenamiento se conirma de forma persistente en todas las fases.
2. En validación, se observa una mejora con la incorporación de la Fase 1, con un recall de 0.7027 y AUC 0.9150 sin embargo, la caída de las métricas en validación es de importancia (promedio 0.1748), continua con una generalización no adecuada.

## 9. Resumen Final Random Forest

In [29]:
# Mostrar resumen final
df_final = pd.read_csv(f"{OUTPUT_DIR_REPORTES}cv_summary_RF.csv")

print("================================================================================")
print("  RESUMEN Random Forest - CROSS VALIDATION")
print("================================================================================")
print(df_final.to_string(index=False))

  RESUMEN Random Forest - CROSS VALIDATION
      modelo fase  accuracy_val_mean  accuracy_val_std  precision_val_mean  precision_val_std  recall_val_mean  recall_val_std  f1_val_mean  f1_val_std  roc_auc_val_mean  roc_auc_val_std  accuracy_train_mean  accuracy_train_std  precision_train_mean  precision_train_std  recall_train_mean  recall_train_std  f1_train_mean  f1_train_std  roc_auc_train_mean  roc_auc_train_std
RandomForest   T0             0.7268            0.0109              0.6194             0.0310           0.3923          0.0107       0.4799      0.0123            0.7380           0.0057               1.0000              0.0000                1.0000               0.0000             1.0000            0.0000         1.0000        0.0000              1.0000             0.0000
RandomForest   T1             0.8539            0.0080              0.8425             0.0279           0.6719          0.0118       0.7473      0.0112            0.8964           0.0095               1.00

## 10. Resumen completo de entrenamiento (Todos los algoritmos)

In [30]:
rl_path = "../outputs/models/baseline_RL/cv_summary_RL.csv"
df_rl = pd.read_csv(rl_path)

rf_path = "../outputs/models/baseline_RF/cv_summary_RF.csv"
df_rf = pd.read_csv(rf_path)

df_resumen = pd.concat([df_rl, df_rf], ignore_index=True)

# Guardar tabla de comparación
df_resumen.to_csv(f"../outputs/models/cv_summary_entrenamiento.csv", index=False)

print(f"Resultados guardados en: ../outputs/models/cv_summary_entrenamiento.csv")

Resultados guardados en: ../outputs/models/cv_summary_entrenamiento.csv


## Conclusión

Aunque Random Forest presenta una mejora progresiva del rendimiento con la incorporación de variables, presenta un sobreajuste severo con alta varianza en todas las las fases en el conjunto de entrenamiento y una diferrencia relevante en las métricas con datos de validación. Este comportamiento, junto con un recall inferior al los resultados de la RL, limita su idoneidad para la detección temprana de la deserción estudiantil.